<a href="https://colab.research.google.com/github/pranitakhade/BizCardX-Extracting-Business-Card-Data-with-OCR/blob/main/BizCardX_Extracting_Business_Card_Data_with_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r requirements.txt

In [3]:
%%writefile app.py

import streamlit as st
from PIL import Image
import numpy as np
import easyocr
import re
import pandas as pd
import sqlite3
from streamlit_option_menu import option_menu

def preprocess_image(image_path):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    image = image.convert("L")

    # Convert the grayscale image to a NumPy array
    image_np = np.array(image)

    # Ensure the image is in 3-channel format (grayscale to RGB)
    if len(image_np.shape) == 2:
        image_np = np.stack((image_np,) * 3, axis=-1)

    # Apply thresholding to binarize the image (adjust the threshold as needed)
    threshold = 128
    image_np = (image_np > threshold).astype(np.uint8) * 255

    # Convert the NumPy array back to a PIL image
    processed_image = Image.fromarray(image_np)

    return processed_image

# Function to extract and process data from extracted text (replace with actual processing code)
def extract_and_process_data(extracted_text):
    l = []
    for item in extracted_text:
        l.append(item[1])  # item[1] contains the extracted text

    patterns = {
        "Email_Address": r".*@.*com",
        "Website_URL": r"(?i).*www.*com",
        "Card_Holder_Name": l[0],
        "Designation": l[1],
        "Mobile_Numbers": r"(?:\+\d{1,3}-\d{3}-\d{4}|\d{3}-\d{3}-\d{4})",
        "Area": r"(\d+\s+[A-Za-z\s_]+),\s*",
        "City": r",\s*([^,;]+)",
        "State": r"\b([A-Za-z]+)\s+\d{6,7}",
        "Pincode": r"\b\d{6,7}\b"
    }

    results = {key: [] for key in patterns}
    results["Company_Name"] = " ".join(l[-1:-4:-2][::-1])  # Join elements in reverse order
    results["Company_Name"] = results["Company_Name"].split(" ")  # Split the joined string
    if any(any(char.isdigit() for char in item) for item in results['Company_Name']) and any("sun" in item.lower() for item in l):
        results["Company_Name"] = 'Sun Electricals'
    if any(any(char.isdigit for char in item) for item in results['Company_Name']) and any("borcelle" in item.lower() for item in l):
        results["Company_Name"] = 'Borcelle Airlines'

    for item in l:
        text = item

        for key, pattern in patterns.items():
            matches = re.findall(pattern, text)
            if matches:
                results[key].extend(matches)

    # Append the last city to the State if necessary
    if len(results['City']) > 1 and not results['State']:
        last_city = results['City'].pop()
        results['State'].append(last_city)

    if not results['State']:
        results['State'] = ['TamilNadu']

    results['Company_Name'] = ' '.join(results['Company_Name'])
    results['Mobile_Numbers'] = [' / '.join(results['Mobile_Numbers'])]

    df = pd.DataFrame(results)

    df['Email_Address'] = df['Email_Address'].str.replace(" com", ".com")
    df['Company_Name'] = df['Company_Name'].str.title()
    df['City'] = df['City'].str.title()
    df['Card_Holder_Name'] = df['Card_Holder_Name'].str.title()
    df['Designation'] = df['Designation'].str.title()
    df['Website_URL'] = df['Website_URL'].str.replace(r'www(?!\.)', 'www.', regex=True)
    df['Website_URL'] = df['Website_URL'].str.lower()
    df['Email_Address'] = df['Email_Address'].str.lower()
    df['Company_Name'] = df['Company_Name'].str.replace(" ", "")

    return df

# Main Streamlit app
def main():
    st.set_page_config(
        page_title=" Extracting Business Card Data with OCR ",
        layout="wide",
        initial_sidebar_state="expanded")

    # Streamlit UI
    st.title("BizCardX: Extracting Business Card Data with OCR")

    selected = option_menu(None, ["Upload & Extract", "ALTER", "DELETE"],
                          icons=["cloud-upload", "pencil-square", "trash"],
                          default_index=0,
                          orientation="horizontal",
                          styles={"nav-link": {"font-size": "20px", "text-align": "center", "margin": "-2px", "--hover-color": "#6495ED"},
                                  "icon": {"font-size": "20px"},
                                  "container": {"max-width": "6000px"},
                                  "nav-link-selected": {"background-color": "#6495ED"}})


    conn = sqlite3.connect("OCR.db")

    conn.execute('''
        CREATE TABLE IF NOT EXISTS Business_Card (
            Email_Address VARCHAR(255) UNIQUE,
            Website_URL VARCHAR(255),
            Card_Holder_Name VARCHAR(255),
            Designation VARCHAR(255),
            Mobile_Numbers VARCHAR(255),
            Area VARCHAR(255),
            City VARCHAR(255),
            State VARCHAR(255),
            Pincode VARCHAR(255),
            Company_Name VARCHAR(255)
        );
    ''')

    # Commit the changes
    conn.commit()

    df = None

    if selected == "Upload & Extract":
        # Path to the uploaded business card image
        image_path = st.file_uploader("Upload Business Card", type=['png', 'jpg', 'jpeg'])

        if image_path is not None:
            # Perform image processing (replace with actual code)
            processed_image = preprocess_image(image_path)

            # Create an EasyOCR reader
            reader = easyocr.Reader(["en"])

            # Extract text from the processed image
            extracted_text = reader.readtext(np.array(processed_image))

            col1,col2,col3 = st.columns([1,2,1],gap='small')

            with col2:
                st.image(image_path, caption="Uploaded Image", use_column_width=True)

            df = extract_and_process_data(extracted_text)

            if st.button("EXTRACT DATA"):
                st.header("Extracted Information")
                st.write(df)

            if st.button("UPLOAD DATA"):
                conn = sqlite3.connect("OCR.db")  # Replace with your database file
                cursor = conn.cursor()

                sql = "INSERT INTO Business_Card (Email_Address, Website_URL, Card_Holder_Name, Designation, Mobile_Numbers, Area, City, State, Pincode, Company_Name) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"

                for index, row in df.iterrows():
                    values = (row['Email_Address'], row['Website_URL'], row['Card_Holder_Name'], row['Designation'], row['Mobile_Numbers'], row['Area'], row['City'], row['State'], row['Pincode'], row['Company_Name'])

                    try:
                        cursor.execute(sql, values)
                        conn.commit()
                        st.success("Data Inserted")
                    except sqlite3.IntegrityError:
                        st.write(":blue[Data already present]")

                cursor.execute("SELECT Email_Address, Website_URL, Card_Holder_Name, Designation, Mobile_Numbers, Area, City, State, Pincode, Company_Name FROM Business_Card")
                data = cursor.fetchall()
                updated_data = pd.DataFrame(data, columns=["Email Address", "Website URL", "Card Holder Name", "Designation", "Mobile Number", "Area", "City", "State", "Pincode", "Company Name"])
                st.write(updated_data)

    if selected == "ALTER":
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT Card_Holder_Name FROM Business_Card")
            result = cursor.fetchall()
            business_cards = {}

            for row in result:
                business_cards[row[0]] = row[0]
            options = ["None"] + list(business_cards.keys())

            selected_card = st.selectbox("Select a Card:", options)
            if selected_card == "None":
                st.write("No Card Selected.")
            else:
                st.markdown("#### Alter Any Data Below")
                cursor.execute("SELECT Email_Address, Website_URL, Card_Holder_Name, Designation, Mobile_Numbers, Area, City, State, Pincode, Company_Name FROM Business_Card WHERE Card_Holder_Name = ?", (selected_card,))
                result = cursor.fetchone()
                Email_Address = st.text_input("Email_Address", result[0])
                Website_URL = st.text_input("Website_URL", result[1])
                Card_Holder_Name = st.text_input("Card_Holder_Name", result[2])
                Designation = st.text_input("Designation", result[3])
                Mobile_Numbers = st.text_input("Mobile_Numbers", result[4])
                Area = st.text_input("Area", result[5])
                City = st.text_input("City", result[6])
                State = st.text_input("State", result[7])
                Pincode = st.text_input("Pincode", result[8])
                Company_Name = st.text_input("Company_Name", result[9])

                if st.button(":blue[Commit changes to DB]"):
                    cursor.execute("UPDATE Business_Card SET Company_Name=?, Card_Holder_Name=?, Designation=?, Mobile_Numbers=?, Email_Address=?, Website_URL=?, Area=?, City=?, State=?, Pincode=? WHERE Card_Holder_Name = ?", (Company_Name, Card_Holder_Name, Designation, Mobile_Numbers, Email_Address, Website_URL, Area, City, State, Pincode, selected_card))
                    conn.commit()
                    st.success("Information updated in the database successfully.")

                    cursor.execute("SELECT Company_Name, Card_Holder_Name, Designation, Mobile_Numbers, Email_Address, Website_URL, Area, City, State, Pincode FROM Business_Card")
                    data1 = cursor.fetchall()
                    updated_df = pd.DataFrame(data1,columns=["Company_Name", "Card_Holder", "Designation", "Mobile_Number", "Email", "Website", "Area", "City", "State", "Pin_Code"])
                    st.write(updated_df)

        except Exception as e:
            st.warning("There is no data available in the database or an error occurred: " + str(e))

    if selected == "DELETE":
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT Card_Holder_Name FROM Business_Card")
            result = cursor.fetchall()
            business_cards = {}
            for row in result:
                business_cards[row[0]] = row[0]
            options = ["None"] + list(business_cards.keys())
            selected_card = st.selectbox("Select a card:", options)

            if selected_card == "None":
                st.write("No card selected.")
            else:
                st.write(f"# Are You Sure, You want to Delete :green[**{selected_card}'s**] Card Details")
                st.write("If 'NOT', Please change Card to 'None'")

                # Add a condition to control the "Yes, sure" button's visibility
                if st.button(":green[Yes, sure]"):

                    # Delete the card details from the database
                    cursor.execute("DELETE FROM Business_Card WHERE Card_Holder_Name=?", (selected_card,))
                    conn.commit()
                    st.success("Business card info. deleted from the database")

                    cursor.execute("SELECT Company_Name, Card_Holder_Name, Designation, Mobile_Numbers, Email_Address, Website_URL, Area, City, State, Pincode FROM Business_Card")
                    data2 = cursor.fetchall()
                    updated_df = pd.DataFrame(data2, columns=["Company_Name", "Card_Holder", "Designation", "Mobile_Number", "Email", "Website", "Area", "City", "State", "Pin_Code"])
                    st.write(updated_df)

        except Exception as e:
            st.warning("There is no data available in the database or an error occurred: " + str(e))


if __name__ == "__main__":
    main()


Writing app.py


In [4]:
!wget -q -O - ipv4.icanhazip.com

34.125.166.32


In [6]:
!streamlit run /content/app.py & npx localtunnel -p 8501

[..................] / rollbackFailedOptional: verb npm-session c77d91e3716d1f4


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.166.32:8501

npx: installed 22 in 3.207s
your url is: https://fine-sides-fold.loca.lt
  Stopping...
^C
